# 코딩 전 생각

In [ ]:
""""""""""
0. 10001개의 리스트(가사 단어의 가능한 길이)를 담는 이중 리스트 두 개(W,RW)를 선언한다.

1. words의 단어들과 역순 단어들을 각각 W와 RW의 단어 길이에 해당하는 위치에 담고 W와 RW의 모든 리스트를 정렬한다.

2-1. 각 쿼리에 대해, ?가 접미어에 있을 경우 ?를 모두 'a'로 바꾸어 bisect_left로 W 리스트에서 해당 단어가 들어갈 인덱스(A)를 구한다.
이어서 이번에는 모든 ?를 'z'로 바꾸어 bisect_right로 W 리스트에서 해당 단어가 들어갈 인덱스(B)를 구한다.
answer 리스트에 b-a값을 append한다.
만약 ?가 접두어에 있을 경우 쿼리의 문자열을 뒤집은 뒤 RW 리스트에서 인덱스를 구한다.

2-2. 각 쿼리에 대해, ?가 접미어에 있을 경우 이진 탐색을 통해 ?가 시작되는 인덱스 idx를 찾는다.
W의 해당 쿼리의 길이에 해당하는 리스트에서 다시 한번 이진탐색을 수행하여
인덱스전까지의 단어가 쿼리의 그것과 일치하는 단어의 개수를 구해 answer에 담는다.
?가 접두어에 붙었을 경우 RW에 대해서 위의 동작을 수행한다.
""""""""""

# 내 답안 - a,z를 활용해 bisect 모듈 사용

In [ ]:
from bisect import bisect_left, bisect_right
def solution(words, queries):
    answer = []
    W = [[] for _ in range(10001)]
    RW = [[] for _ in range(10001)]
    
    for word in words:
        W[len(word)].append(word)
        RW[len(word)].append(word[::-1])
    for i in range(1, 10001):
        W[i].sort()
        RW[i].sort()
        
    for query in queries:
        if query[-1] == '?':
            A = bisect_left(W[len(query)], query.replace('?', 'a'))
            B = bisect_right(W[len(query)], query.replace('?', 'z'))
        else:
            A = bisect_left(RW[len(query)], query[::-1].replace('?', 'a'))
            B = bisect_right(RW[len(query)], query[::-1].replace('?', 'z'))
        answer.append(B-A)
    return answer

# 내 답안 - 이진 탐색 연속 구현

In [ ]:
def solution(words, queries):
    def binary_search(query):
        s, e = 0, len(query) - 1
        idx = -1
        while s <= e:
            mid = (s + e) // 2
            if query[mid] == '?':
                idx = mid
                e = mid - 1
            else:
                s = mid + 1
        return idx
    def binary_search_count(query, W):
        idx = binary_search(query)
        s, e = 0, len(W) - 1
        lidx, ridx = -1, -1
        while s <= e:
            mid = (s + e) // 2
            if W[mid][:idx] >= query[:idx]:
                if W[mid][:idx] == query[:idx] and (mid == 0 or W[mid - 1][:idx] < query[:idx]):
                    lidx = mid
                    break
                e = mid - 1
            else:
                s = mid + 1
        s, e = 0, len(W) - 1
        while s <= e:
            mid = (s + e) // 2
            if W[mid][:idx] > query[:idx]:
                e = mid - 1
            else:
                if W[mid][:idx] == query[:idx] and (mid == len(W) - 1 or W[mid + 1][:idx] > query[:idx]):
                    ridx = mid
                    break
                s = mid + 1
        if ridx == -1:
            return 0
        else:    
            return ridx - lidx + 1
    answer = []
    W = [[] for _ in range(10001)]
    RW = [[] for _ in range(10001)]
    
    for word in words:
        W[len(word)].append(word)
        RW[len(word)].append(word[::-1])
    for i in range(1, 10001):
        W[i].sort()
        RW[i].sort()
    for query in queries:
        if query[-1] == '?' and query[0] == '?':
            answer.append(len(W[len(query)]))
        elif query[-1] == '?':
            answer.append(binary_search_count(query, W[len(query)]))
        else:
            answer.append(binary_search_count(query[::-1], RW[len(query)]))
    return answer